In [ ]:
import pandas as pd
import os
from PIL import Image

In [ ]:
# import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import time
import os
import PIL.Image as Image
from IPython.display import display

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.get_device_name(device))


### splited train_labels.csv to train and validation csv file 
download the dataset from https://www.kaggle.com/c/cs-t0828-2020-hw1/data
put the "training_data" file and "testing file" all in this repo

In [ ]:
# def split_data():
#     data = pd.read_csv("train_label.csv")
#     data = data.iloc[:300]
#     labels = data.groupby("label")
#     np.random.seed(10)
#     for i, label in enumerate(data.label.unique()):
#         randomList = list(range(len(labels.get_group(label))))
#         np.random.shuffle(randomList)
#         trainPart = labels.get_group(label).iloc[randomList][:int(len(randomList)*0.8)]
#         testPart = labels.get_group(label).iloc[randomList][int(len(randomList)*0.8):]
#         if (i == 0):
#             trainAll = trainPart
#             testAll = testPart
#         else:
#             trainAll = pd.concat([trainAll, trainPart], axis=0)
#             testAll = pd.concat([testAll, testPart], axis=0)
#     trainAll.to_csv("train.csv",index=False)
#     testAll.to_csv("valid.csv",index=False)
# split_data()

### mount image to image files according to there label_index if needed 
do it twice for train.csv and valid.csv

In [ ]:
# rootPath = "training_data/training_data//"
# df = pd.read_csv('valid.csv')
# for i, x in df.iterrows():
#     print(x.id, x.label)
#     img = Image.open(os.path.join(rootPath, str(x.id).zfill(6)+'.jpg'))
#     if not os.path.isdir(f"./car_dataset/test/{x.label_index}"):
#         os.mkdir(f"./car_dataset/test/{x.label_index}")
#     img.save(f"./car_dataset/test/{x.label_index}/{str(x.id).zfill(6)}.jpg")

### data augmentation, data normalization, and load datas

In [ ]:
dataset_dir = "./car_dataset/"
batch_size = 16
train_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.RandomRotation(15),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
test_tfms = transforms.Compose([transforms.Resize((400, 400)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

dataset = torchvision.datasets.ImageFolder(root=dataset_dir+"train", transform=train_tfms)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

dataset2 = torchvision.datasets.ImageFolder(root=dataset_dir+"test", transform=test_tfms)
testloader = torch.utils.data.DataLoader(dataset2, batch_size=batch_size, shuffle=False)


## Function to train and evaluation

In [ ]:
def train_model(model, criterion, optimizer, scheduler, n_epochs=5):
    losses = []
    accuracies = []
    test_accuracies = []
    # set the model to train mode initially
    model.train()
    for epoch in range(n_epochs):
        since = time.time()
        running_loss = 0.0
        running_correct = 0.0
        for i, data in enumerate(trainloader, 0):
            # get the inputs and assign them to cuda
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # calculate the loss/acc later
            running_loss += loss.item()
            running_correct += (labels == predicted).sum().item()
        epoch_duration = time.time()-since
        epoch_loss = running_loss/len(trainloader)
        epoch_acc = 100/batch_size*running_correct/len(trainloader)
        print("Epoch %s, duration: %d s, loss: %.4f, acc: %.4f" % (epoch+1, epoch_duration, epoch_loss, epoch_acc))
        losses.append(epoch_loss)
        accuracies.append(epoch_acc)
        # switch the model to eval mode to evaluate on test data
        model.eval()
        test_acc = eval_model(model)
        test_accuracies.append(test_acc)
        # re-set the model to train mode after validating
        model.train()
        scheduler.step(test_acc)
        since = time.time()
    print('Finished Training')
    return model, losses, accuracies, test_accuracies


In [ ]:
def eval_model(model):
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for i, data in enumerate(testloader, 0):
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)
            outputs = model_ft(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_acc = 100.0 * correct / total
    print('Accuracy of the network on the test images: %d %%' % (
        test_acc))
    return test_acc


### train model

In [ ]:
# load pretrain model
model_ft = models.resnet152(pretrained=True)
num_ftrs = model_ft.fc.in_features
# replace the last fc layer with an untrained one (requires grad by default)
model_ft.fc = nn.Linear(num_ftrs, 196)
model_ft = model_ft.to(device)
# loss function and optimzer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.01, momentum=0.9)
lrscheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, threshold=0.9)


In [ ]:
model_ft, training_losses, training_accs, test_accs = train_model(model_ft, criterion, optimizer, lrscheduler, n_epochs=1)

In [ ]:
# save model parameters
torch.save({"epoch": 10, 
           "model_state_dict": model_ft.state_dict(),
            "optimizer_state_dict": optimizer.state_dict()},
           "model/resnet152.pth")

### load parameter without retraining again

In [ ]:
# load the model trained parameters
checkpoint = torch.load("model/resnet152.pth")
model_ft.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])

### inference

In [ ]:
# using classes.txt find accrodingly class name
classes = pd.read_table("classes.txt", names=["class"])

In [ ]:
def find_classes(dir):
    classes = os.listdir(dir)
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    return classes, class_to_idx


pp, c_to_idx = find_classes(dataset_dir+"train")

In [ ]:
# switch the model to evaluation mode to make dropout and batch norm work in eval mode
model_ft.eval()

# transforms for the input image
loader = transforms.Compose([transforms.Resize((400, 400)),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
results = pd.DataFrame(data={'id': [], 'label': []})
# load test image from file
for file in os.listdir("testing_data/testing_data/"):
    try:

        image = Image.open(os.path.join('testing_data/testing_data', file))
        image = image.convert('RGB')
        image = loader(image).float()
    except RuntimeError:

        print(file)
        break
    image = torch.autograd.Variable(image, requires_grad=True)
    image = image.unsqueeze(0)
    image = image.to(device)
    output = model_ft(image)
    conf, predicted = torch.max(output.data, 1)
    label = classes.loc[int(pp[predicted.item()])]['class']
    idx = os.path.splitext(file)[0]
    # print image id, label_index of image file, accordingly class name, predicted confidence(probability)
    print(idx, label, predicted.item(), conf)
    results = results.append({'id': idx, 'label': label}, ignore_index=True)
    

In [ ]:
# save to csv
results.to_csv('results.csv', index=False, header=1)